<center><img src="http://alacip.org/wp-content/uploads/2014/03/logoEscalacip1.png" width="500"></center>


<center> <h1>Curso: Introducción al Python</h1> </center>

<br></br>

* Profesor:  <a href="http://www.pucp.edu.pe/profesor/jose-manuel-magallanes/" target="_blank">Dr. José Manuel Magallanes, PhD</a> ([jmagallanes@pucp.edu.pe](mailto:jmagallanes@pucp.edu.pe))<br>
    - Profesor del **Departamento de Ciencias Sociales, Pontificia Universidad Católica del Peru**.<br>
    - Senior Data Scientist del **eScience Institute** and Visiting Professor at **Evans School of Public Policy and Governance, University of Washington**.<br>
    - Fellow Catalyst, **Berkeley Initiative for Transparency in Social Sciences, UC Berkeley**.


## Parte 4:  Data Cleaning en Python

El pre procesamiento de datos es la parte más tediosa del proceso de investigación.

Esta primera parte delata diversos problemas que se tienen con los datos reales que están en la web, como la que vemos a continuación:

In [2]:
import IPython
wikiLink="https://en.wikipedia.org/wiki/List_of_freedom_indices" 
iframe = '<iframe src=' + wikiLink + ' width=700 height=350></iframe>'
IPython.display.HTML(iframe)

Recuerda inspeccionar la tabla para encontrar algun atributo que sirva para su descarga. De ahí, continua.

In [3]:
# antes instala'beautifulsoup4'
# es posible que necesites salir y volver a cargar notebook

import pandas as pd

wikiTables=pd.read_html(wikiLink,header=0,flavor='bs4',attrs={'class': 'wikitable sortable'})

In [4]:
# cuantas tenemos?
len(wikiTables)

1

Hasta aquí todo parece bien. Como solo hay uno, lo traigo y comienzo a verificar 'suciedades'.

In [5]:
DF=wikiTables[0]

#primera mirada
DF.head()

,Country,Freedom in the World 2019[10],2019 Index of Economic Freedom[11],2019 Press Freedom Index[3],2018 Democracy Index[13]
0,Afghanistan,not free,mostly unfree,difficult situation,authoritarian regime
1,Albania,partly free,moderately free,noticeable problems,hybrid regime
2,Algeria,not free,repressed,difficult situation,authoritarian regime
3,Andorra,free,NaN,satisfactory situation,NaN
4,Angola,not free,mostly unfree,noticeable problems,authoritarian regime


La limpieza requiere estrategia. Lo primero que salta a la vista, son los _footnotes_ que están en los títulos:

In [6]:
DF.columns

Index(['Country', 'Freedom in the World 2019[10]',
       '2019 Index of Economic Freedom[11]', '2019 Press Freedom Index[3]',
       '2018 Democracy Index[13]'],
      dtype='object')

In [7]:
# aqui ves que pasa cuando divido cada celda usando el caracter '['
[element.split('[') for element in DF.columns]

[['Country'],
 ['Freedom in the World 2019', '10]'],
 ['2019 Index of Economic Freedom', '11]'],
 ['2019 Press Freedom Index', '3]'],
 ['2018 Democracy Index', '13]']]

In [8]:
# Te das cuenta que te puedes quedar con el primer elemento cada vez que partes:
[element.split('[')[0] for element in DF.columns]

['Country',
 'Freedom in the World 2019',
 '2019 Index of Economic Freedom',
 '2019 Press Freedom Index',
 '2018 Democracy Index']

También hay que evitar espacios en blanco:

In [9]:
outSymbol=' ' 
inSymbol=''
[element.split('[')[0].replace(outSymbol,inSymbol) for element in DF.columns]

['Country',
 'FreedomintheWorld2019',
 '2019IndexofEconomicFreedom',
 '2019PressFreedomIndex',
 '2018DemocracyIndex']

Los números también molestan, pero están en diferentes sitios. Mejor intentemos expresiones regulares:

In [10]:
import re  # debe estar instalado.

# espacios: \\s+
# uno o mas numeros \\d+
# bracket que abre \\[
# bracket que cierra \\]

pattern='\\s+|\\d+|\\[|\\]'
nothing=''

#substituyendo 'pattern' por 'nothing':
[re.sub(pattern,nothing,element) for element in DF.columns]

['Country',
 'FreedomintheWorld',
 'IndexofEconomicFreedom',
 'PressFreedomIndex',
 'DemocracyIndex']

Ya tengo nuevos titulos de columna (headers)!!

In [11]:
newHeaders=[re.sub(pattern,nothing,element) for element in DF.columns]

Preparemos los cambios:

In [12]:
list(zip(DF.columns,newHeaders))

[('Country', 'Country'),
 ('Freedom in the World 2019[10]', 'FreedomintheWorld'),
 ('2019 Index of Economic Freedom[11]', 'IndexofEconomicFreedom'),
 ('2019 Press Freedom Index[3]', 'PressFreedomIndex'),
 ('2018 Democracy Index[13]', 'DemocracyIndex')]

In [13]:
# veamos los cambios:
{old:new for old,new in zip(DF.columns,newHeaders)}

{'Country': 'Country',
 'Freedom in the World 2019[10]': 'FreedomintheWorld',
 '2019 Index of Economic Freedom[11]': 'IndexofEconomicFreedom',
 '2019 Press Freedom Index[3]': 'PressFreedomIndex',
 '2018 Democracy Index[13]': 'DemocracyIndex'}

Uso un dict por si hubieses querido cambiar solo algunas columnas:

In [14]:
changes={old:new for old,new in zip(DF.columns,newHeaders)}
DF.rename(columns=changes,inplace=True)

In [15]:
# ahora tenemos:
DF.head()

,Country,FreedomintheWorld,IndexofEconomicFreedom,PressFreedomIndex,DemocracyIndex
0,Afghanistan,not free,mostly unfree,difficult situation,authoritarian regime
1,Albania,partly free,moderately free,noticeable problems,hybrid regime
2,Algeria,not free,repressed,difficult situation,authoritarian regime
3,Andorra,free,NaN,satisfactory situation,NaN
4,Angola,not free,mostly unfree,noticeable problems,authoritarian regime


Las columnas son categorías, veamos si todas se han escrito de la manera correcta:

In [16]:
DF.FreedomintheWorld.value_counts()

free           87
partly free    62
not free       55
Name: FreedomintheWorld, dtype: int64

In [17]:
DF.IndexofEconomicFreedom.value_counts()

mostly unfree      64
moderately free    59
mostly free        29
repressed          22
free                6
Name: IndexofEconomicFreedom, dtype: int64

In [18]:
DF.PressFreedomIndex.value_counts()

noticeable problems       73
difficult situation       54
satisfactory situation    28
very serious situation    19
good situation            15
Name: PressFreedomIndex, dtype: int64

In [19]:
DF.DemocracyIndex.value_counts()

flawed democracy        55
authoritarian regime    53
hybrid regime           39
full democracy          20
Name: DemocracyIndex, dtype: int64

Pues hasta aquí está conforme. Veamos otro caso.

_______

In [20]:
idhCol="https://www.datosmacro.com/idh/colombia" 
iframe = '<iframe src=' + idhCol + ' width=700 height=350></iframe>'
IPython.display.HTML(iframe)

Luego de inspeccionar la tabla, podemos traerla:

In [21]:
import pandas as pd

webTable=pd.read_html(idhCol,header=0,flavor='bs4',attrs={'id': 'tb0'})

In [22]:
len(webTable)

1

In [23]:
idhColT=webTable[0]
idhColT

,Fecha,IDH,Ranking IDH
0,2017,747,90º
1,2016,747,89º
2,2015,742,160º
3,2014,738,158º
4,2013,735,157º
5,2012,725,162º
6,2011,725,159º
7,2010,719,160º
8,2009,715,161º
9,2008,710,161º


El problema es que se borraron los decimales. Como se ve en la web, estos tenían una coma en vez de un punto. A esta altura podemos eliminarlo, o buscar si durante el proceso de colección se puede mejorar esto; dale una mirada a la función:

In [24]:
?pd.read_html

Siguiendo las instrucciones escibimos:

In [25]:
idhColT=pd.read_html(idhCol,header=0,flavor='bs4',attrs={'id': 'tb0',},
                      thousands=None, decimal=',')[0]
idhColT

,Fecha,IDH,Ranking IDH
0,2017,0.747,90º
1,2016,0.747,89º
2,2015,0.742,160º
3,2014,0.738,158º
4,2013,0.735,157º
5,2012,0.725,162º
6,2011,0.725,159º
7,2010,0.719,160º
8,2009,0.715,161º
9,2008,0.710,161º


El ranking no es un numero, pues el símbolo lo evita, eliminemoslo ([revisar](https://ascii.cl/es/codigos-html.htm)):

In [26]:
idhColT.loc[:,'Ranking IDH']=idhColT.loc[:,'Ranking IDH'].str.replace(chr(186),"")
idhColT

,Fecha,IDH,Ranking IDH
0,2017,0.747,90
1,2016,0.747,89
2,2015,0.742,160
3,2014,0.738,158
4,2013,0.735,157
5,2012,0.725,162
6,2011,0.725,159
7,2010,0.719,160
8,2009,0.715,161
9,2008,0.710,161


______


Traigamos una nueva tabla:

In [27]:
idhCol2='https://es.wikipedia.org/wiki/Anexo:Departamentos_de_Colombia_por_IDH'
iframe = '<iframe src=' + idhCol2 + ' width=700 height=350></iframe>'
IPython.display.HTML(iframe)

Aparentemente sabemos qué hacer:

In [28]:
idhColT2=pd.read_html(idhCol2,header=0,flavor='bs4',attrs={'class': 'sortable',},
                       thousands=' ', decimal=',')[0]
idhColT2

,Entidad,IDH,Población[3]​,País Comparable[4]​
0,Bogotá,0.792,8181047,Turquía
1,Valle del Cauca,0.771,4755760,Granada
2,San Andrés y Providencia,0.770,78413,Sri Lanka
3,Atlántico,0.766,2546138,Bosnia-Herzegovina
4,Quindío,0.765,574960,Bosnia-Herzegovina
5,Meta,0.758,1016672,Brasil
6,Santander,0.758,2 090 854,Brasil
7,Caldas,0.757,993870,Líbano
8,Cundinamarca,0.754,2804238,Argelia
9,Antioquia,0.752,6690977,Ecuador


Aparentemente, sólo Boyacá tenía espacios en blanco.

En este caso, el primer problema es que los miles marcados con _espacios_ no desaparecieron. Eso se debe a que en el html están señalados como **& nbsp;**. De ahi que:

In [29]:
idhColT2=pd.read_html(idhCol2,header=0,flavor='bs4',attrs={'class': 'sortable',},
                       thousands=chr(160), decimal=',')[0]
idhColT2

,Entidad,IDH,Población[3]​,País Comparable[4]​
0,Bogotá,0.792,8 181 047,Turquía
1,Valle del Cauca,0.771,4 755 760,Granada
2,San Andrés y Providencia,0.770,78 413,Sri Lanka
3,Atlántico,0.766,2 546 138,Bosnia-Herzegovina
4,Quindío,0.765,574 960,Bosnia-Herzegovina
5,Meta,0.758,1 016 672,Brasil
6,Santander,0.758,2 090 854,Brasil
7,Caldas,0.757,993 870,Líbano
8,Cundinamarca,0.754,2 804 238,Argelia
9,Antioquia,0.752,6 690 977,Ecuador


Pues, Boyacá es ahora el problema. Eso lo resolveremos fuera de la llamada:

In [30]:
idhColT2.iloc[:,2]=idhColT2.iloc[:,2].str.replace("\s","")
idhColT2

,Entidad,IDH,Población[3]​,País Comparable[4]​
0,Bogotá,0.792,8181047,Turquía
1,Valle del Cauca,0.771,4755760,Granada
2,San Andrés y Providencia,0.770,78413,Sri Lanka
3,Atlántico,0.766,2546138,Bosnia-Herzegovina
4,Quindío,0.765,574960,Bosnia-Herzegovina
5,Meta,0.758,1016672,Brasil
6,Santander,0.758,2090854,Brasil
7,Caldas,0.757,993870,Líbano
8,Cundinamarca,0.754,2804238,Argelia
9,Antioquia,0.752,6690977,Ecuador


Los nombres de columnas necesitan tratamiento, podríamos usar lo que ya vimos:

In [31]:
import re

[re.sub(pattern,nothing,element) for element in idhColT2.columns]

['Entidad', 'IDH', 'Población\u200b', 'PaísComparable\u200b']

O mejorar el patrón:

In [32]:
pattern2='\\s+|\\d+|\\[|\\]|\\u200b'
[re.sub(pattern2,nothing,element) for element in idhColT2.columns]

['Entidad', 'IDH', 'Población', 'PaísComparable']

Pero esta vez, hay _footnotes_ con texto, cuando antes sólo tenía números, de ahi que jueguemos simple:

In [33]:
[element.split('[')[0].replace(" ","") for element in idhColT2.columns]

['Entidad', 'IDH', 'Población', 'PaísComparable']

Cambiemos con esto los _headers_:

In [34]:
idhColT2.columns=[element.split('[')[0].replace(" ","") for element in idhColT2.columns]
idhColT2

,Entidad,IDH,Población,PaísComparable
0,Bogotá,0.792,8181047,Turquía
1,Valle del Cauca,0.771,4755760,Granada
2,San Andrés y Providencia,0.770,78413,Sri Lanka
3,Atlántico,0.766,2546138,Bosnia-Herzegovina
4,Quindío,0.765,574960,Bosnia-Herzegovina
5,Meta,0.758,1016672,Brasil
6,Santander,0.758,2090854,Brasil
7,Caldas,0.757,993870,Líbano
8,Cundinamarca,0.754,2804238,Argelia
9,Antioquia,0.752,6690977,Ecuador


Sucede algo similar con los contenidos de las columnas (vease 'Región Amazónica'). De ahí que:

In [35]:
idhColT2.Entidad=[element.split('[')[0] for element in idhColT2.Entidad]
idhColT2

,Entidad,IDH,Población,PaísComparable
0,Bogotá,0.792,8181047,Turquía
1,Valle del Cauca,0.771,4755760,Granada
2,San Andrés y Providencia,0.770,78413,Sri Lanka
3,Atlántico,0.766,2546138,Bosnia-Herzegovina
4,Quindío,0.765,574960,Bosnia-Herzegovina
5,Meta,0.758,1016672,Brasil
6,Santander,0.758,2090854,Brasil
7,Caldas,0.757,993870,Líbano
8,Cundinamarca,0.754,2804238,Argelia
9,Antioquia,0.752,6690977,Ecuador


In [60]:
idhColT2.Población=pd.to_numeric(idhColT2.Población)

In [62]:
idhColT2.describe()

,IDH,Población
count,34.000000,3.400000e+01
mean,0.726059,2.931455e+06
std,0.035606,8.480695e+06
min,0.624000,4.344600e+04
25%,0.702500,4.055090e+05
50%,0.730000,1.052915e+06
75%,0.753500,1.804138e+06
max,0.792000,4.983473e+07


____

[Ir a inicio](#beginning)

_____

**AUSPICIO**: 

* El desarrollo de estos contenidos ha sido posible gracias al grant del Berkeley Initiative for Transparency in the Social Sciences (BITSS) at the Center for Effective Global Action (CEGA) at the University of California, Berkeley


<center>
<img src="https://www.bitss.org/wp-content/uploads/2015/07/bitss-55a55026v1_site_icon.png" style="width: 200px;"/>
</center>

* Este curso cuenta con el auspicio de:


<center>
<img src="https://www.python.org/static/img/psf-logo@2x.png" style="width: 500px;"/>
</center>



**RECONOCIMIENTO**


EL Dr. Magallanes agradece a la Pontificia Universidad Católica del Perú, por su apoyo en la participación en la Escuela ALACIP.

<center>
<img src="https://dci.pucp.edu.pe/wp-content/uploads/2014/02/Logotipo_colores-290x145.jpg" style="width: 400px;"/>
</center>


El autor reconoce el apoyo que el eScience Institute de la Universidad de Washington le ha brindado desde el 2015 para desarrollar su investigación en Ciencia de Datos.

<center>
<img src="https://escience.washington.edu/wp-content/uploads/2015/10/eScience_Logo_HR.png" style="width: 500px;"/>
</center>

<br>
<br>